# Figures

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import norm

paper_width = 8.3
paper_margin = 0

sns.set_theme(
    context="paper",
    style="ticks",
    palette="colorblind",
    color_codes=True,
    rc={
        "figure.figsize": (paper_width - 2 * paper_margin, 4 * (paper_width - 2 * paper_margin) / 6),
        "figure.titlesize": 12,  # "large",
        "axes.titlesize": 10,  # 9.6,
        "axes.labelsize": 10,  # 9.6,
        "legend.fontsize": 10,  # 8.8,
        "legend.title_fontsize": 10,  # 9.6,
        "xtick.labelsize": 8,  # 8.8,
        "xtick.major.size": 7,  # 4.8,
        "xtick.minor.size": 5,  # 3.2,
        "ytick.labelsize": 8,  # 8.8,
        "ytick.major.size": 7,  # 4.8,
        "ytick.minor.size": 5,  # 3.2,
        "font.size": 10,  # 9.6,
        "font.family": 'lmodern',
        "text.usetex": True,
        "text.latex.preamble": [
            r"\usepackage{amsmath}",
            r"\usepackage{lmodern}",
        ],
    },
)

rng = np.random.default_rng(seed=123)

## Figure 1

Created with BioRender

## Figure 2

In [ ]:
cell_data = pd.read_csv("../data/kuett_catena_2022/sectioning/cell_data.csv")
cell_slice_data = pd.read_csv("../data/kuett_catena_2022/sectioning/cell_slice_data.csv")

In [ ]:
mu_r = cell_data.loc[cell_data["sectioning_axis_name"] == "z", "proj_cell_radius_um"].mean()
sigma_r = cell_data.loc[cell_data["sectioning_axis_name"] == "z", "proj_cell_radius_um"].std()

t = cell_slice_data["section_thickness_um"].unique()
r = rng.normal(loc=mu_r, scale=sigma_r, size=len(cell_data.index))
s = np.ceil(2 * r[:, np.newaxis] / t[np.newaxis, :])

def s_dash(k):
    n1 = np.floor((t[np.newaxis, :] - ((2 * r[:, np.newaxis]) % t[np.newaxis, :])) / k) + 1
    n2 = np.ceil(((2 * r[:, np.newaxis]) % t[np.newaxis, :]) / k) - 1
    return (n1 * s + (n2 * (s + 1))) / (n1 + n2)

In [ ]:
def plot_volume(ax=None):
    if ax is None:
        ax = plt.gca()
    ax.axis("off")

In [ ]:
def plot_cell_radius(ax=None, num=1000):
    if ax is None:
        ax = plt.gca()
    x_r = np.linspace(
        cell_data.loc[cell_data["sectioning_axis_name"] == "z", "proj_cell_radius_um"].min(),
        cell_data.loc[cell_data["sectioning_axis_name"] == "z", "proj_cell_radius_um"].max(),
        num,
    )
    y_r = norm.pdf(x_r, mu_r, sigma_r)
    sns.kdeplot(
        data=cell_data[cell_data["sectioning_axis_name"] == "z"],
        x="proj_cell_radius_um",
        label=r"$r_z = \sqrt{\frac{A_z}{\pi}}$",
        legend=False,
        ax=ax,
    )
    ax.plot(x_r, y_r, label=r"$R \sim \mathcal{N}(\mu_r, \sigma_r)$", ls="--")
    ax.text(
        0.9,
        0.5,
        f"""$\\hat{{\\mu}}_r = {mu_r:.3f}$
            $\\hat{{\\sigma}}_r = {sigma_r:.3f}$""",
        transform=ax.transAxes,
        ha="right",
        va="center",
    )
    ax.set_title(r"Cell radius (circular approximation)")
    ax.set_xlabel(r"Radius [$\mu m$]")
    ax.set_ylabel(r"Density")
    ax.legend()

In [ ]:
def plot_cell_section_span(ax=None):
    if ax is None:
        ax = plt.gca()
    data = pd.concat(
        {
            r"z-sectioning": cell_slice_data[cell_slice_data["sectioning_axis_name"] == "z"].groupby(
                ["section_thickness_um", "section_offset_um", "cell_id"]
            ).size().reset_index(name="cell_section_span").groupby(
                ["section_thickness_um", "cell_id"]
            )["cell_section_span"].mean().reset_index(name="cell_section_span").drop(columns="cell_id"),

            r"y-sectioning": cell_slice_data[cell_slice_data["sectioning_axis_name"] == "y"].groupby(
                ["section_thickness_um", "section_offset_um", "cell_id"]
            ).size().reset_index(name="cell_section_span").groupby(
                ["section_thickness_um", "cell_id"]
            )["cell_section_span"].mean().reset_index(name="cell_section_span").drop(columns="cell_id"),

            r"x-sectioning": cell_slice_data[cell_slice_data["sectioning_axis_name"] == "x"].groupby(
                ["section_thickness_um", "section_offset_um", "cell_id"]
            ).size().reset_index(name="cell_section_span").groupby(
                ["section_thickness_um", "cell_id"]
            )["cell_section_span"].mean().reset_index(name="cell_section_span").drop(columns="cell_id"),

            r"$S = \lceil 2 R t^{-1} \rceil$": pd.DataFrame(data=s, columns=t).melt(
                var_name="section_thickness_um", value_name="cell_section_span"
            ),

            r"$S'(2)$": pd.DataFrame(data=s_dash(2), columns=t).melt(
                var_name="section_thickness_um", value_name="cell_section_span"
            ),

            r"$\lim_{k \to 0}~S'(k)$": pd.DataFrame(data=s_dash(1e-12), columns=t).melt(
                var_name="section_thickness_um", value_name="cell_section_span"
            )
        },
        names=["label", ""],
    ).reset_index()
    data["section_thickness_um"] = data["section_thickness_um"].astype("category")
    g = sns.lineplot(
        data=data,
        x="section_thickness_um",
        y="cell_section_span",
        hue="label",
        marker="o",
        ci=None,
        ax=ax,
    )
    g.get_legend().set_title(None)
    ax.set_title(r"Cell section span")
    ax.set_xlabel(r"Section thickness [$\mu m$]")
    ax.set_ylabel(r"Number of sections")

In [ ]:
def plot_cell_miss_probab(ax=None):
    if ax is None:
        ax = plt.gca()
    data = pd.DataFrame(data=1 / s_dash(1e-12), columns=t).melt(
        var_name="section_thickness_um", value_name="cell_miss_probab"
    )
    data["section_thickness_um"] = data["section_thickness_um"].astype("category")
    g = sns.kdeplot(
        data=data,
        x="cell_miss_probab",
        hue="section_thickness_um",
        ax=ax,
    )
    g.get_legend().set_title(r"Section thickness [$\mu m$]")    
    ax.set_title(r"Cell miss probability")
    ax.set_xlabel(r"$P($cell miss in next section$) = \lim_{k \to 0}~S'(k)^{-1}$")
    ax.set_ylabel(r"Density")
    ax.set_xlim(0, 1)

In [ ]:
fig, axes = plt.subplot_mosaic(
    """AB
       CD""",
)
plot_volume(ax=axes["A"])
plot_cell_radius(ax=axes["B"])
plot_cell_section_span(ax=axes["C"])
plot_cell_miss_probab(ax=axes["D"])
plt.tight_layout()
plt.show()